<a href="https://colab.research.google.com/github/steevschmidt/NEC-220.87-Methods/blob/main/2022_HEA_220_87_Methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Run this cell (Shift+Enter) as a prerequisite for using Gradio UI in Google Colab,
# then make sure to perform "Runtime > Restart session" for the change to take effect!

# Note: tested with gradio==5.17.1

!pip install -U gradio

# Highest hourly load estimation from meter data

In this section we construct and demonstrate our highest hourly load estimation step by step, using some example CSV data.

In the next pane, we start by loading the CSV data into a pandas dataframe (pandas is the standard Python library for manipulating tabular data).

Naively, one would expect the energy usage data from a single meter to be either hourly or 15-minute level interval data. In reality, the data is sometimes mixed (some hours have just the hourly sum value, some have the 15-minute subintervals), some values may be missing (for an entire hour, or maybe just for some 15-minute intervals), duplicate values for the same interval (with the same or different value) may exist, and quite often the 15-minute-interval data is in fact hourly (all subinterval values are the same; e.g. the hourly sum value / 4). The algorithm presented here is supposed to deal with all such cases in a single pass by deriving the period (interval length for scaling values into hourly level) and character of the data (real or fake hourly) from the data itself.

Take a closer look at the example data below to observe some peculiarities that will highlight the exact working of our algorithm:
 * There are four identical values for 15-minute subintervals at hour 14:00. When all values from an hour are all the same, they are interpreted either as normal hourly data (in normal case when there's just one value) or as "fake 15-minute interval" (in case there are multiple values, but all are same). Both cases will lead to the applying of a multiplicative factor (1.3) for hourly-level data later on.
 * There are only three values for 15-minute subintervals at hour 15:00. Since not all the values of this hour are the same, they will be interpreted as true 15-minute interval data, i.e. the multiplicative factor for hourly-level data will not be applied later on.
 * There are two identical values for the same interval 16:00:00. This will be interpreted as if there was just a single value (the higher value) for that interval because the timestamps are not different. Because there is a distinct value for 16:15:00, the period (data resolution) for that hour will still be 4 (15-minute interval).
 * There is just one value for 17:00:00. Because there are no other values, that value will be interpreted as period 1 (hourly).

In [39]:
import pandas as pd
import numpy  as np
from io import StringIO

sample_csv = """DateTime,kWh
"2023-11-25 14:00:00",0.81
"2023-11-25 14:15:00",0.81
"2023-11-25 14:30:00",0.81
"2023-11-25 14:45:00",0.81
"2023-11-25 15:00:00",0.78
"2023-11-25 15:15:00",0.86
"2023-11-25 15:30:00",0.78
"2023-11-25 15:45:00",0.81
"2023-11-25 16:00:00",0.75
"2023-11-25 16:00:00",0.77
"2023-11-25 16:15:00",0.67
"2023-11-25 17:00:00",3.23
"""

df = pd.read_csv(StringIO(sample_csv), parse_dates=["DateTime"])
print("Raw input data (mixed hourly/15-minute interval):\n", df, "\n")


Raw input data (mixed hourly/15-minute interval):
               DateTime   kWh
0  2023-11-25 14:00:00  0.81
1  2023-11-25 14:15:00  0.81
2  2023-11-25 14:30:00  0.81
3  2023-11-25 14:45:00  0.81
4  2023-11-25 15:00:00  0.78
5  2023-11-25 15:15:00  0.86
6  2023-11-25 15:30:00  0.78
7  2023-11-25 15:45:00  0.81
8  2023-11-25 16:00:00  0.75
9  2023-11-25 16:00:00  0.77
10 2023-11-25 16:15:00  0.67
11 2023-11-25 17:00:00  3.23 



## Group data into hourly intervals and calculate per-hour data quality metrics

The first step is to group the raw meter data by (top of) hour from each timestamp, find the max (interval-level) kWh within each hour, and also the count of unique meter values and the count of unique timestamps within the same hour.

Following that, we also add a "period" column to indicate whether we classify the hour's data as "15-minute-interval-level" (no matter if fake or real) or "hourly-level". Values are only assumed as "hourly-level" when there is a single unique timestamp within that hour; otherwise they are assumed as "15-minute-level".

In [40]:
df.set_index('DateTime', inplace=True, drop=False)
df_hourly = df.groupby(pd.Grouper(freq='h', level='DateTime')).agg({'kWh': ['max', 'nunique'], 'DateTime': 'nunique'})
df_hourly.columns = df_hourly.columns.map('_'.join)
df_hourly['period'] = np.where(df_hourly['DateTime_nunique'] == 1, 1, 4)
df_hourly = df_hourly.reset_index()

print("After hourly aggregation, maximum subinterval value (kWh_max) calculation, and fake 15-minute data (kWh_nunique) and period (hourly=1 or 15-minute resolution=4) detection:\n", df_hourly, "\n")

After hourly aggregation, maximum subinterval value (kWh_max) calculation, and fake 15-minute data (kWh_nunique) and period (hourly=1 or 15-minute resolution=4) detection:
              DateTime  kWh_max  kWh_nunique  DateTime_nunique  period
0 2023-11-25 14:00:00     0.81            1                 4       4
1 2023-11-25 15:00:00     0.86            3                 4       4
2 2023-11-25 16:00:00     0.77            3                 2       4
3 2023-11-25 17:00:00     3.23            1                 1       1 



## Calculate maximum load for each hour; extrapolate from 15-minute maxima; apply adjustment factor

Finally, we use the intermediate metrics obtained above to estimate the final maximum hourly load (power draw) values:
 * we multiply any found 15-minute kWh usage maximum by 4 (based on determined "period")
 * we multiply any hourly-level data (either real hourly data or fake 15-minute interval data) by 1.3 (based on determined count of unique values within the hour)

In [41]:
df_hourly['kW_max_adj'] = np.where(df_hourly['kWh_nunique'] == 1, df_hourly['kWh_max'] * df_hourly['period'] * 1.3, df_hourly['kWh_max'] * df_hourly['period'])

print("After applying extrapolation using period and the factor 1.3 for hourly-level data:\n", df_hourly, "\n")

After applying extrapolation using period and the factor 1.3 for hourly-level data:
              DateTime  kWh_max  kWh_nunique  DateTime_nunique  period  \
0 2023-11-25 14:00:00     0.81            1                 4       4   
1 2023-11-25 15:00:00     0.86            3                 4       4   
2 2023-11-25 16:00:00     0.77            3                 2       4   
3 2023-11-25 17:00:00     3.23            1                 1       1   

   kW_max_adj  
0       4.212  
1       3.440  
2       3.080  
3       4.199   



## Estimate maximum hourly load over the entire analyzed period

The largest of the determined hourly maxima is our estimated highest hourly load (which is then used as a parameter for the remaining panel capacity calculation):

In [42]:
peak_hourly_load_kW = df_hourly['kW_max_adj'].max()

print(f"Expected peak hourly load = {peak_hourly_load_kW:.2f} kW")

Expected peak hourly load = 4.21 kW


# Remaining panel capacity calculation

We estimate the remaining panel capacity by subtracting the estimated peak hourly load multiplied by a safety factor from the current panel capacity (provided in amps):

In [43]:
panel_size_A = 150
panel_voltage_V = 240
current_capacity_kW = panel_size_A * panel_voltage_V / 1000
remaining_capacity_kW = current_capacity_kW - 1.25 * peak_hourly_load_kW

print(f"Estimated remaining panel capacity = {remaining_capacity_kW:.2f} kW")

Estimated remaining panel capacity = 30.73 kW


# Wrap up

The following pane contains all the calculations described above wrapped into two reusable Python functions which can be used standalone (with the required preinstalled pandas and numpy packages):

In [44]:
import pandas as pd
import numpy as np

def get_peak_hourly_load(df : pd.DataFrame, return_df : bool = False) -> float | pd.DataFrame:
    """Estimates the peak hourly load in kW from meter values.

    Arguments:
        df:
            Input meter values, supplied as a pandas dataframe with the following columns:
                "DateTime" (the measurement interval's start in format "YYYY-MM-DD HH:MM:00"),
                "kWh" (the measured meter value in kilowatt-hours)
            The dataframe may contain a mix of hourly and 15-minute interval values.
        return_df:
            Whether to return the peak hourly load estimated over the entire dataframe (default)
            or for the individual hours.

    Returns:
        Either a float value for the estimated peak hourly load in kW
        or the dataframe with estimated peak hourly loads for the individual hours
    """
    df.set_index('DateTime', inplace=True, drop=False)
    df_hourly = df.groupby(pd.Grouper(freq='h', level='DateTime')).agg({'kWh': ['max', 'nunique'], 'DateTime': 'nunique'})
    df_hourly.columns = df_hourly.columns.map('_'.join)
    df_hourly['period'] = np.where(df_hourly['DateTime_nunique'] == 1, 1, 4)
    df_hourly['kW_max_adj'] = np.where(df_hourly['kWh_nunique'] == 1, df_hourly['kWh_max'] * df_hourly['period'] * 1.3, df_hourly['kWh_max'] * df_hourly['period'])
    return df_hourly if return_df else df_hourly['kW_max_adj'].max()

def get_remaining_panel_capacity(peak_hourly_load_kW : float, panel_size_A : int, panel_voltage_V = 240) -> float:
    """Estimates the remaining panel capacity in kW from panel size and peak hourly load.

    Arguments:
        peak_hourly_load_kW:
            Estimated peak hourly load in kilowatts, see get_peak_hourly_load
        panel_size_A:
            Current panel size in amperes (amps)
        panel_voltage_V:
            Current panel voltage in volts (default: 240V)
    Returns:
        A float value for the remaining electric panel capacity in kW, according to NEC-220.87
    """
    return panel_size_A * panel_voltage_V / 1000 - 1.25 * peak_hourly_load_kW

In [45]:
print(f"Expected peak hourly load = {get_peak_hourly_load(df):.2f} kW")
print(f"Estimated remaining panel capacity = {get_remaining_panel_capacity(get_peak_hourly_load(df), 150):.2f} kW")

Expected peak hourly load = 4.21 kW
Estimated remaining panel capacity = 30.73 kW


# Gradio UI

This simple UI allows the user to upload a CSV file with their meter data, outputs the calculated peak hourly load and remaining panel capacity and displays a chart with average/max observed and peak loads. Note that no input validation is performed on the uploaded data (yet).

In order be able to start the Gradio GUI below, make sure that you have first run the first "pip install" cell of this notebook to install Gradio (and restarted session) and after that also have run the above "Wrap up" cell which contains the definitions of the panel tool Python functions (because the UI uses them).

In [59]:
import gradio as gr
import pandas as pd
from io import StringIO

import warnings
from altair.utils.deprecation import AltairDeprecationWarning
warnings.filterwarnings("ignore", category=AltairDeprecationWarning)

#import altair as alt
#alt.data_transformers.enable("vegafusion")

sample_csv = """DateTime,kWh
"2023-11-25 14:00:00",0.81
"2023-11-25 14:15:00",0.81
"2023-11-25 14:30:00",0.81
"2023-11-25 14:45:00",0.81
"2023-11-25 15:00:00",0.78
"2023-11-25 15:15:00",0.86
"2023-11-25 15:30:00",0.78
"2023-11-25 15:45:00",0.81
"2023-11-25 16:00:00",0.75
"2023-11-25 16:00:00",0.77
"2023-11-25 16:15:00",0.67
"2023-11-25 17:00:00",3.23
"""

def clear_inputs():
    return None, 150, 240, gr.update(visible=False), gr.update(visible=False), gr.update(visible=False)

def process_inputs(temp_file, panel_size_A, panel_voltage_V):
    if temp_file.startswith("DateTime"):
      df = pd.read_csv(StringIO(temp_file), parse_dates=["DateTime"])
    else:
      df = pd.read_csv(temp_file.name, parse_dates=["DateTime"])

    peak_df = get_peak_hourly_load(df, return_df = True)
    peak_hourly_load_kW = peak_df['kW_max_adj'].max()
    remaining_panel_capacity_kW = get_remaining_panel_capacity(peak_hourly_load_kW, panel_size_A, panel_voltage_V)

    # As a bonus, draw a simple plot of the estimated indvidual peak hourly loads,
    # along with the calculated overall peak hourly load as a horizontal line above them

    peak_df['hour'] = peak_df.index.hour
    
    df_hourly_max  = peak_df.groupby('hour').agg({'kW_max_adj': 'max'}).reset_index(drop=False)
    df_hourly_peak = peak_df.groupby('hour').agg({'kW_max_adj': 'first'}).reset_index(drop=False)
    df_hourly_peak['kW_max_adj'] = peak_hourly_load_kW
    
    df_hourly = pd.concat([df_hourly_peak, df_hourly_max], ignore_index=True)
    df_hourly['stat'] = ['peak'] * len(df_hourly_peak) + ['max'] * len(df_hourly_max)

    return (
        gr.update(value=peak_hourly_load_kW, visible=True), 
        gr.update(value=remaining_panel_capacity_kW, visible=True),
        gr.update(value=df_hourly, visible=True)
    )

with gr.Blocks() as demo:
    gr.Markdown("### Upload Meter Data and Panel Information")
    
    with gr.Row():
        with gr.Column(scale=1):
            file_input = gr.UploadButton("Click to upload meter data in CSV format (DateTime, kWh)", file_types=['.csv'], file_count="single")
            panel_capacity = gr.Number(label="Current panel capacity in amps", value=150)
            panel_voltage = gr.Number(label="Current panel voltage", value=240)
            
            with gr.Row():
                submit_btn = gr.Button("Calculate", variant="primary")
                clear_btn = gr.Button("Clear")
        
        with gr.Column(scale=1):
            peak_load = gr.Number(label="Peak hourly load in kW", precision=2, visible=False)
            remaining_capacity = gr.Number(label="Remaining panel capacity in kW", precision=2, visible=False)
            hourly_load_plot = gr.LinePlot(label="Hourly loads", x="hour", y="kW_max_adj", color='stat', width=400, height=200, interactive=False, visible=False)
    
    submit_btn.click(fn=process_inputs, inputs=[file_input, panel_capacity, panel_voltage], outputs=[peak_load, remaining_capacity, hourly_load_plot])
    clear_btn.click(fn=clear_inputs, inputs=[], outputs=[file_input, panel_capacity, panel_voltage, peak_load, remaining_capacity, hourly_load_plot])

demo.launch()
#demo.launch(debug=True)

* Running on local URL:  http://127.0.0.1:7903

To create a public link, set `share=True` in `launch()`.


# Additional information / original HEA implementation

For further reference, here's the query used in HEA SmartAudit to compute sv_highestIntervalKW (same as highestHourlyKWAdj from the algorithm described above, but with an addition of the estimated PV output on top of the hourly-aggregated net meter value):

In [12]:
%%sql
SELECT MAX(s.max_value * s.period * IF(s.cnt=1,1.3,1)) kW FROM (
    SELECT STR_TO_DATE(e.SM_DATETIME, '%Y-%m-%d %H') sm_datetime, COUNT(DISTINCT e.value) cnt, MAX(e.value + IFNULL(pv.value / e.period, 0)) max_value, MAX(e.period) period
    FROM smart_meter_electric e
    LEFT JOIN pv_output pv ON (e.USER_ID = pv.USER_ID AND STR_TO_DATE(e.SM_DATETIME, '%Y-%m-%d %H') = pv.CM_DATETIME)
    WHERE
        e.sm_datetime >= NOW() - INTERVAL 1 YEAR and e.YEAR IN (YEAR(NOW()), YEAR(NOW())-1)
        AND e.USER_ID = :userId
    GROUP BY STR_TO_DATE(e.SM_DATETIME, '%Y-%m-%d %H')
) s

UsageError: Cell magic `%%sql` not found.


For further reference, here's how we currently compute sv_remainingPanelCapacity based on sv_highestIntervalKW obtained from the above query:

In [11]:
%%javascript
    var actualPanelSize = integerValue(getValue('Electrification', 'be_PanelSize'));

    var panelSizes;
    if (actualPanelSize > 0)
    {
        panelSizes = [ actualPanelSize ];
    }
    else
    {
        panelSizes = STD_PANEL_SIZES.paramValue || '';
        panelSizes = panelSizes.split(',');
    }

    var arr = [];
    for (var i = 0; i < panelSizes.length; i++)
    {
        // [Panel Capacity in kW] = [Panel size in AMPs] * 240 Volts / 1000
        // [Remaining capacity in kW] = [Panel Capacity] - 1.25 * [Peak power in kW]

        var capacityKW = new java.lang.Integer(panelSizes[i]) * 240 / 1000;
        var remainingCapacityKW = capacityKW - 1.25 * sv_highestIntervalKW;

        if (remainingCapacityKW > 0 && arr.length < 3)
        {
            arr.push({ "panelSize": new java.lang.Integer(panelSizes[i]), "kW": remainingCapacityKW });
        }
    }

    var html = '';
    if (arr.length > 0)
    {
      html = '<table style="width:370px">';
      for (var i = 0; i < arr.length; i++)
      {
        html += '<tr><td class="sectionsubtitle" style="width:292px">Remaining capacity for a ' + arr[i].panelSize + ' amp panel:</td><td class="sectionsubtitle" style="text-align:right">' + formatNumber(arr[i].kW, '###.##') + '&nbs
      }
      html += '</table>';
    }

    return html

<IPython.core.display.Javascript object>